In [1]:
# !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/902/829/305/GCA_902829305.4_PGI_SPEXI_v6/GCA_902829305.4_PGI_SPEXI_v6_genomic.fna.gz
# !mv GCA_902829305.4_PGI_SPEXI_v6_genomic.fna.gz genomic.fna.gz
# !gunzip genomic.fna.gz

# ZDNABERT

Взято из репозитория https://github.com/mitiau/Z-DNABERT/blob/main/ZDNA-prediction.ipynb

## Install dependecies and define helper functions

In [2]:
%pip install transformers
%pip install biopython

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
# from google.colab import drive, files
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

2025-06-11 19:44:30.474930: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 19:44:30.677022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749660270.749942    7031 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749660270.769860    7031 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749660270.922309    7031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

# def stitch_np_seq(np_seqs, pad=16):
#     total_len = sum(len(seq) for seq in np_seqs) - pad * (len(np_seqs) - 1)
#     if total_len <= 0:
#         return np.array([], dtype=np.float32)
#     res = np.empty(total_len, dtype=np.float32)
#     pos = 0
#     for i, seq in enumerate(np_seqs):
#         if i > 0:
#             pos = max(0, pad)
#         length = len(seq)
#         res[pos:pos + length] = seq
#         pos += length
#     return res

## Select model and parameters

In [5]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [6]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'


In [7]:
#!gdown $model_id
# !gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
# !gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
# !gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
# !gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


#!mkdir 6-new-12w-0
#!mv pytorch_model.bin 6-new-12w-0/
# !mv config.json 6-new-12w-0/
# !mv special_tokens_map.json 6-new-12w-0/
# !mv tokenizer_config.json 6-new-12w-0/
# !mv vocab.txt 6-new-12w-0/

In [8]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Upload fasta files for prediction

In [9]:
# uploaded = files.upload()
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [10]:
with open('genomic.fna', 'rb') as f:
  uploaded = {'genomic': f.read()}

## Predict and save raw outputs

In [11]:
out = []
for key in uploaded.keys():
    print(key)
    out.append(key)
    result_dict = {}
    for seq_record in SeqIO.parse(StringIO(BytesIO(uploaded[key]).read().decode('UTF-8')), 'fasta'):
        kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
        seq_pieces = split_seq(kmer_seq)
        print(seq_record.name)
        out.append(seq_record.name)
        with torch.no_grad():
            preds = []
            for seq_piece in tqdm(seq_pieces):
                input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
                outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1],axis = -1)[0,:,1]
                preds.append(outputs.cpu().numpy())
        result_dict[seq_record.name] = stitch_np_seq(preds)



        labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
        # print('  start     end')
        out.append('  start     end')
        for label in range(1, max_label+1):
            candidate = np.where(labeled == label)[0]
            candidate_length = candidate.shape[0]
            if candidate_length>minimum_sequence_length:
                # print('{:8}'.format(candidate[0]), '{:8}'.format(candidate[-1]))
                out.append('{:8}'.format(candidate[0]) + '{:8}'.format(candidate[-1]))

    with open(key + '.preds.pkl',"wb") as fh:
      pickle.dump(result_dict, fh)
    print()

with open('text_predictions.txt',"w") as fh:
    for item in out:
        fh.write("%s\n" % item)


genomic
LR824602.2


100%|█████████████████████████████████████| 39415/39415 [22:25<00:00, 29.29it/s]


LR824603.2


100%|█████████████████████████████████████| 34753/34753 [19:44<00:00, 29.34it/s]


LR824604.2


...

100%|███████████████████████████████████████| 6175/6175 [03:30<00:00, 29.39it/s]


CADEPN050000003.1


100%|███████████████████████████████████████| 3731/3731 [02:07<00:00, 29.34it/s]


CADEPN050000004.1


100%|███████████████████████████████████████| 1424/1424 [00:48<00:00, 29.35it/s]


## преобразуем в bed

In [14]:
import pandas as pd
from tqdm import tqdm

with open('text_predictions.txt', 'r') as f:
    scaffold = ''
    lines = f.readlines()


results = pd.DataFrame(columns=['Scaffold', 'Start', 'End'], index=range(len(lines)))

for i, line in tqdm(enumerate(lines), total=len(lines)):
    line = line.strip()
    if line == 'genomic' or line.startswith('start'):
        continue
    if line.startswith('L') or line.startswith('C'):
        scaffold = line
        continue
    
    if len(line.split()) == 1:
        len_s = len(line) // 2
        s, e = line[:len_s], line[len_s:]
    else:
        s, e = line.split()
    s, e = int(s), int(e)
    results.loc[i] = [scaffold, s, e]
results = results.dropna()
results.index = range(len(results))
results

100%|██████████████████████████████████| 96103/96103 [00:02<00:00, 36095.29it/s]


,Scaffold,Start,End
0,LR824602.2,3527,3552
1,LR824602.2,3657,3690
2,LR824602.2,4050,4070
3,LR824602.2,4738,4755
4,LR824602.2,4888,4899
...,...,...,...
96027,CADEPN050000004.1,703806,703818
96028,CADEPN050000004.1,703840,703850
96029,CADEPN050000004.1,704104,704118
96030,CADEPN050000004.1,704236,704251


In [15]:
results.to_csv('zdnabert.bed', index=False, header=False, sep='\t')

# ZHUNT

In [17]:
!chmod a+x zhunt3-alan.c
!gcc zhunt3-alan.c -lm -o zhunt3 -std=c90 -O3

zhunt3-alan.c: In function ‘user_regret’:
zhunt3-alan.c:303:7: warning: ‘gets’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
  303 |       gets( tempstr );
      |       ^~~~
In file included from /usr/include/stdio.h:894,
                 from zhunt3-alan.c:27:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:240:1: note: declared here
  240 | gets (char *__str)
      | ^~~~
zhunt3-alan.c: In function ‘analyze_zscore’:
zhunt3-alan.c:642:18: warning: assignment to ‘char *’ from ‘int’ makes pointer from integer without a cast []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wint-conversion-Wint-conversion]8;;]
  642 |       antisyn[i] = strdup( tempstr );
      |                  ^
zhunt3-alan.c: In function ‘user_regret’:
zhunt3-alan.c:303:7: warning: ignoring return value of ‘gets’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-O

In [18]:
!./zhunt3 12 8 12 genomic.fna

dinucleotides 12
min/max 8 12
min/max 8 12
operating on genomic.fna
calculating zscore
opening genomic.fna
inputting sequence
opening genomic.fna.Z-SCORE

 run time=21079 sec
use min/max 8 12
analyzing_zscore
opening genomic.fna.Z-SCORE


In [19]:
!head genomic.fna.Z-SCORE

genomic.fna 450885468 8 12
1 17 16  29.697  19.432 4.936124e-01 tagagnacacctaacc   ASASSASASASASASA
2 18 16  32.232  17.601 1.715825e-01 agagnacacctaacct   SASAASASASSASASA
3 19 16  30.179  15.406 4.233678e-01 gagnacacctaaccta   ASSASASASASAASAS
4 24 20  32.282  15.829 1.669393e-01 agnacacctaacctaaccta   SAASASASASASASASASAS
5 21 16  30.129  15.080 4.305851e-01 gnacacctaacctaac   SASASASASASASASA
6 24 18  31.749  11.334 2.205216e-01 nacacctaacctaaccta   ASASASASASASASASAS
7 23 16  30.322  14.140 4.030077e-01 acacctaacctaacct   SASASASASASASASA
8 24 16  31.282  12.553 2.747381e-01 cacctaacctaaccta   ASASASASASASASAS
9 25 16  33.585  22.954 7.417036e-02 acctaacctaacctaa   SASASASASASASASA


In [20]:
segs = {}
cur = 1

for record in SeqIO.parse("genomic.fna", "fasta"):
    name, seq = record.id, str(record.seq)
    sz = len(seq)
    segs[name] = (cur, cur + sz - 1)
    cur += sz
segs

{'LR824602.2': (1, 19549462),
 'LR824603.2': (19549463, 36786807),
 'LR824604.2': (36786808, 53377778),
 'LR824605.2': (53377779, 69618174),
 'LR824606.2': (69618175, 85746721),
 'LR824607.2': (85746722, 101840773),
 'LR824608.2': (101840774, 117669678),
 'LR824609.2': (117669679, 133493842),
 'LR824610.2': (133493843, 149286655),
 'LR824611.2': (149286656, 164928677),
 'LR824612.2': (164928678, 180356271),
 'LR824613.2': (180356272, 195684983),
 'LR824614.2': (195684984, 210877553),
 'LR824615.2': (210877554, 225964022),
 'LR824616.2': (225964023, 241001821),
 'LR824617.2': (241001822, 255829335),
 'LR824618.2': (255829336, 270461710),
 'LR824619.2': (270461711, 284996514),
 'LR824620.2': (284996515, 299253309),
 'LR824621.2': (299253310, 313461087),
 'LR824622.2': (313461088, 327516395),
 'LR824623.2': (327516396, 340814947),
 'LR824624.2': (340814948, 352963919),
 'LR824625.2': (352963920, 364941904),
 'LR824626.2': (364941905, 375556839),
 'LR824627.2': (375556840, 386138837),
 'LR

In [ ]:
import pandas as pd

result = pd.read_csv("genomic.fna.Z-SCORE", skiprows=1, names=["start","end","len","1","2","score","seq","3"], delim_whitespace=True)
result = result[result['score'] >= 400]
result

/tmp/ipykernel_7031/910999059.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  result = pd.read_csv("genomic.fna.Z-SCORE", skiprows=1, names=["start","end","len","1","2","score","seq","3"], delim_whitespace=True)


In [2]:
import pandas as pd
from tqdm import tqdm

chunks = pd.read_csv(
    "genomic.fna.Z-SCORE",
    skiprows=1,
    names=["start", "end", "len", "1", "2", "score", "seq", "3"],
    delim_whitespace=True,
    chunksize=10000
)

result = []
for chunk in tqdm(chunks):
    filtered_chunk = chunk[chunk['score'] >= 400]
    result.append(filtered_chunk)

result = pd.concat(result, ignore_index=True)

/tmp/ipykernel_89229/2766162285.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  chunks = pd.read_csv(
45089it [07:09, 105.00it/s]


In [3]:
result

,start,end,len,1,2,score,seq,3
0,3525,3549,24,21.606,23.116,647.6098,ccccggtgtcttctcgcacgtgcg,ASASASASASASASASASASASAS
1,3526,3550,24,21.572,24.304,674.1187,cccggtgtcttctcgcacgtgcgc,ASSASASASASASASASASASASA
2,3527,3551,24,19.621,36.762,8867.7300,ccggtgtcttctcgcacgtgcgcg,ASASASASASASASASASASASAS
3,3528,3552,24,19.620,36.613,8883.3840,cggtgtcttctcgcacgtgcgcgc,SASASASASASASASASASASASA
4,3529,3553,24,18.210,46.418,77328.8400,ggtgtcttctcgcacgtgcgcgcg,ASASASASASASASASASASASAS
...,...,...,...,...,...,...,...,...
1672827,450884767,450884783,16,20.863,37.866,1626.4230,tgctgcgcccgcgctt,ASSASASASASASASA
1672828,450884768,450884784,16,21.605,37.802,648.0815,gctgcgcccgcgctta,ASASASASASASASAS
1672829,450884769,450884785,16,20.784,38.517,1800.5180,ctgcgcccgcgcttat,SASASASASASASASA
1672830,450884770,450884786,16,21.605,37.652,648.0815,tgcgcccgcgcttata,ASASASASASASASAS


In [10]:
from Bio import SeqIO

segs = []
cur = 1

for record in SeqIO.parse("genomic.fna", "fasta"):
    name, seq = record.id, str(record.seq)
    sz = len(seq)
    segs.append((name, cur, cur+sz-1))
    cur += sz
segs[:10]

[('LR824602.2', 1, 19549462),
 ('LR824603.2', 19549463, 36786807),
 ('LR824604.2', 36786808, 53377778),
 ('LR824605.2', 53377779, 69618174),
 ('LR824606.2', 69618175, 85746721),
 ('LR824607.2', 85746722, 101840773),
 ('LR824608.2', 101840774, 117669678),
 ('LR824609.2', 117669679, 133493842),
 ('LR824610.2', 133493843, 149286655),
 ('LR824611.2', 149286656, 164928677)]

In [11]:
from tqdm.auto import tqdm

segs_p = 0

with open('genomic.fna.Z-SCORE', 'r') as inp, open('zhunt.bed', 'w') as out:
    for line in tqdm(inp, total=450885469):
        if line.startswith('genomic.fna'):
            continue
        if float(line.split()[5]) < 400:
            continue

        start = int(line.split()[0])
        end = int(line.split()[1])

        while segs_p < len(segs) and segs[segs_p][2] < start:
            segs_p += 1
        if segs_p == len(segs):
            continue

        name, l, r = segs[segs_p]
        if l <= start <= end <= r:
            out.write(f'{name}\t{start}\t{end}\n')

!head 'zhunt.bed'

  0%|          | 0/383924861 [00:00<?, ?it/s]

LR824602.2	3525	3549
LR824602.2	3526	3550
LR824602.2	3527	3551
LR824602.2	3528	3552
LR824602.2	3529	3553
LR824602.2	3530	3554
LR824602.2	3531	3553
LR824602.2	3532	3554
LR824602.2	3533	3553
LR824602.2	3534	3554


# Квадруплексы

In [12]:
%pip install biopython
import re
from Bio import SeqIO
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pattern_plus = "(?:G{3,5}[ATGC]{1,7}){3,}G{3,5}"
pattern_minus = "(?:C{3,5}[ATGC]{1,7}){3,}C{3,5}"

In [14]:
fasta_sequence = [el for el in SeqIO.parse("genomic.fna",'fasta')]

In [15]:
results = []

for record in tqdm(fasta_sequence):
    name, sequence = record.id, str(record.seq)
    for m in re.finditer(pattern_plus, sequence, re.IGNORECASE):
      results.append([name, m.start(), m.end(), m.group(0), '', '+'])

    for m in re.finditer(pattern_minus, sequence, re.IGNORECASE):
      results.append([name, m.start(), m.end(), m.group(0), '', '-'])

results = pd.DataFrame(results, columns=['scaffold', 'start', 'end', 'sequence', 'name', 'strand'])
results

  0%|          | 0/35 [00:00<?, ?it/s]

,scaffold,start,end,sequence,name,strand
0,LR824602.2,20909,20941,gggagtccaagggcaagggaaagggtaagggg,,+
1,LR824602.2,52873,52898,gggggacgggagagggttatgtggg,,+
2,LR824602.2,57784,57809,gggggacgggagagggttatgtggg,,+
3,LR824602.2,66403,66428,gggggacgggagagggttatgtggg,,+
4,LR824602.2,94936,94968,GGGCCCACGTGGGAACTGCGGGGAAGAGAGGG,,+
...,...,...,...,...,...,...
32875,CADEPN050000004.1,476142,476167,CCCGGTCCCAAAATCCCCTCGACCC,,-
32876,CADEPN050000004.1,479644,479669,cccacataaccctctcccgtccccc,,-
32877,CADEPN050000004.1,483131,483156,cccaatccttccccgccccaatccc,,-
32878,CADEPN050000004.1,537392,537407,cccaccctcccgccc,,-


In [16]:
results.to_csv('quadruplex.bed', index=False, header=False, sep='\t')